In [1]:
pip install transformers torch rdflib requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import time
from rdflib import Graph, URIRef, Namespace

In [3]:
with open("texte_a_analyser.txt", "r", encoding="utf-8") as f:
    text_source = f.read()

# Découpage : on sépare par les doubles sauts de ligne (\n\n) et on enlève les vides
blocs = [b.strip() for b in text_source.split('\n') if b.strip()]

print(f"Texte découpé avec succès en {len(blocs)} blocs (paragraphes).")
print(f"Exemple du bloc 1 : {blocs[0]}")

# REBEL

from transformers import pipeline
import pprint

# 1. Chargement du pipeline REBEL
# Le premier lancement téléchargera le modèle (~1.6 Go)
triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

# 2. Fonction minimale pour parser la sortie (String -> Liste de Dictionnaires)
def extract_triplets(text):
    triplets = []
    relation, subject, object_ = '', '', ''
    text = text.strip()
    current = 'x'
    # on enlève juste <s>, <pad>, </s> mais on garde <triplet>, <subj>, <obj>
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(), 'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(), 'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(), 'tail': object_.strip()})
    return triplets


# 3. Exécution sur les blocs
print("Démarrage de l'extraction REBEL...\n")
all_triplets = []

for i, bloc in enumerate(blocs):
    # 1) génération : on veut les ids, pas le texte nettoyé
    out = triplet_extractor(
        bloc,
        max_length=512,
        num_beams=3,
        return_tensors=True,
        return_text=False
    )[0]["generated_token_ids"]

    # 2) décodage manuel en gardant les tokens spéciaux
    raw_output = triplet_extractor.tokenizer.batch_decode(
        [out],
        skip_special_tokens=False
    )[0]

    # Debug si tu veux voir la chaîne brute
    print(f"RAW OUTPUT BLOC {i+1} : {repr(raw_output)}")

    # 3) extraction des triplets
    triplets = extract_triplets(raw_output)
    all_triplets.extend(triplets)

    print(f"Bloc {i+1} : {len(triplets)} triplets extraits.")

print(f"\nTerminé ! Total de triplets trouvés : {len(all_triplets)}")
pprint.pprint(all_triplets)


Texte découpé avec succès en 9 blocs (paragraphes).
Exemple du bloc 1 : Geoffrey Everest Hinton (born 6 December 1947) is an English Canadian cognitive psychologist and computer scientist, most noted for his work on artificial neural networks. Since 2013 he divides his time working for Google (Google Brain) and the University of Toronto. In 2017, he cofounded and became the Chief Scientific Advisor of the Vector Institute in Toronto.


/home/laurine/Documents/M2 IA/KNOWLEDGE ENG./TP3/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:0


Démarrage de l'extraction REBEL...

RAW OUTPUT BLOC 1 : '<s><triplet> Everest Hinton <subj> 6 December 1947 <obj> date of birth <subj> computer scientist <obj> occupation <subj> artificial neural network <obj> field of work <subj> University of Toronto <obj> employer <triplet> artificial neural network <subj> cognitive psychologist <obj> part of</s>'
Bloc 1 : 5 triplets extraits.
RAW OUTPUT BLOC 2 : '<s><triplet> AlexNet <subj> Alex Krizhevsky <obj> discoverer or inventor <triplet> Alex Krizhevsky <subj> AlexNet <obj> notable work</s>'
Bloc 2 : 2 triplets extraits.
RAW OUTPUT BLOC 3 : '<s><triplet> Christopher Longuet-Higgins <subj> University of Edinburgh <obj> employer</s>'
Bloc 3 : 1 triplets extraits.
RAW OUTPUT BLOC 4 : '<s><triplet> Google <subj> DNNresearch Inc. <obj> subsidiary <triplet> DNNresearch Inc. <subj> Google <obj> parent organization</s>'
Bloc 4 : 2 triplets extraits.
RAW OUTPUT BLOC 5 : '<s><triplet> neural network <subj> machine learning <obj> use</s>'
Bloc 5 : 1 tr

In [4]:
def get_dbpedia_uri(text_entity, confidence=0.5, support=20):
    api_url = "https://api.dbpedia-spotlight.org/en/annotate"
    params = {
        "text": text_entity,
        "confidence": confidence,
        "support": support
    }
    headers = {"Accept": "application/json"}
    
    try:
        time.sleep(0.2) 
        response = requests.get(api_url, params=params, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if 'Resources' in data:
                return data['Resources'][0]['@URI']
    except Exception as e:
        print(e)
    
    return None

In [ ]:
g = Graph()
EX = Namespace("http://example.org/")
cache = {}

all_names = set()
for t in all_triplets:
    all_names.add(t['head'])
    all_names.add(t['tail'])

print(f"Recherche URIs pour {len(all_names)} entités...")
for nom in all_names:
    uri = get_dbpedia_uri(nom)
    if uri:
        print(f"{nom} -> {uri}")
        cache[nom] = URIRef(uri)
    else:
        # fallback
        clean_nom = nom.replace(" ", "_")
        local_uri = EX[clean_nom]
        print(f"{nom} -> {local_uri} (Local)")
        cache[nom] = local_uri

print("Ajout des triplets...")
for t in all_triplets:
    s = cache[t['head']]
    o = cache[t['tail']]
    
    # local relation
    rel_clean = t['type'].replace(" ", "_")
    p = EX[rel_clean]
    
    g.add((s, p, o))

print("\n" + "="*40)
print(" RÉSULTAT FINAL (.ttl) ")
print("="*40)
print(g.serialize(format='turtle'))

Recherche URIs pour 23 entités...
Boltzmann machine -> http://dbpedia.org/resource/Boltzmann_machine
artificial neural network -> http://dbpedia.org/resource/Neural_network
Terry Sejnowski -> http://dbpedia.org/resource/Terry_Sejnowski
cognitive psychologist -> http://dbpedia.org/resource/Cognitive_psychology
machine learning -> http://dbpedia.org/resource/Machine_learning
David E. Rumelhart -> http://dbpedia.org/resource/David_Rumelhart
Google -> http://dbpedia.org/resource/Google
David Ackley -> http://example.org/David_Ackley (Local)
Christopher Longuet-Higgins -> http://dbpedia.org/resource/Christopher_Longuet-Higgins
6 December 1947 -> http://example.org/6_December_1947 (Local)
Alex Krizhevsky -> http://example.org/Alex_Krizhevsky (Local)
neural network -> http://dbpedia.org/resource/Neural_network
research papers -> http://example.org/research_papers (Local)
open-access -> http://dbpedia.org/resource/Open_access
Carnegie Mellon University -> http://dbpedia.org/resource/Carnegie_M

In [6]:
# test diff values for conf, support
def test(entities_list):
    scenarios = [
        (0.3, 0),    
        (0.5, 20),  
        (0.8, 100)  
    ]
    sample_entities = list(set(entities_list))[:4] 
    
    for ent in sample_entities:
        for conf, supp in scenarios:
            uri = get_dbpedia_uri(ent, conf, supp)
            res = uri.split('/')[-1] if uri else None
            print(f"{ent}: {uri}")

In [7]:
# build graph
def build_final_graph(triplets):
    g = Graph()
    EX = Namespace("http://example.org/") 
    
    FINAL_CONF = 0.5
    FINAL_SUPP = 20
    
    noms_uniques = set()
    for t in triplets:
        noms_uniques.add(t['head'])
        noms_uniques.add(t['tail'])
        
    test(list(noms_uniques))
        
    uri_cache = {}
    
    for nom in noms_uniques:
        uri = get_dbpedia_uri(nom, FINAL_CONF, FINAL_SUPP)
        if uri:
            uri_cache[nom] = URIRef(uri)
        else:
            clean = nom.replace(" ", "_")
            uri_cache[nom] = EX[clean]

    for t in triplets:
        s = uri_cache[t['head']]
        o = uri_cache[t['tail']]
        
        rel_clean = t['type'].replace(" ", "_").replace("-", "_") # to get local relationships
        p = EX[rel_clean]
        
        g.add((s, p, o))
        
    return g



In [11]:
graph_final = build_final_graph(all_triplets)

Boltzmann machine: http://dbpedia.org/resource/Boltzmann_machine
Boltzmann machine: http://dbpedia.org/resource/Boltzmann_machine
Boltzmann machine: None
artificial neural network: http://dbpedia.org/resource/Artificial_neural_network
artificial neural network: http://dbpedia.org/resource/Neural_network
artificial neural network: http://dbpedia.org/resource/Neural_network
Terry Sejnowski: http://dbpedia.org/resource/Terry_Sejnowski
Terry Sejnowski: http://dbpedia.org/resource/Terry_Sejnowski
Terry Sejnowski: None
cognitive psychologist: http://dbpedia.org/resource/Cognitive_psychology
cognitive psychologist: http://dbpedia.org/resource/Cognitive_psychology
cognitive psychologist: http://dbpedia.org/resource/Cognitive_psychology


In [12]:
print(graph_final.serialize(format='turtle'))

@prefix ns1: <http://example.org/> .

<http://dbpedia.org/resource/Boltzmann_machine> ns1:discoverer_or_inventor <http://dbpedia.org/resource/Terry_Sejnowski>,
        ns1:David_Ackley .

<http://dbpedia.org/resource/Christopher_Longuet-Higgins> ns1:employer <http://dbpedia.org/resource/Edinburgh> .

<http://dbpedia.org/resource/David_Rumelhart> ns1:educated_at <http://dbpedia.org/resource/Carnegie_Mellon_University> .

<http://dbpedia.org/resource/Mount_Everest> ns1:date_of_birth ns1:6_December_1947 ;
    ns1:employer <http://dbpedia.org/resource/Toronto> ;
    ns1:field_of_work <http://dbpedia.org/resource/Neural_network> ;
    ns1:occupation <http://dbpedia.org/resource/Computer_scientist> .

<http://dbpedia.org/resource/Open_access> ns1:subclass_of ns1:research_papers .

<http://dbpedia.org/resource/Google> ns1:subsidiary <http://example.org/DNNresearch_Inc.> .

<http://dbpedia.org/resource/Neural_network> ns1:part_of <http://dbpedia.org/resource/Cognitive_psychology> ;
    ns1:use